In [1]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

1.15.0


In [2]:
CSV_COLUMN_NAMES = ["time"]+['V'+(i+1).astype(np.str) for i in np.array(range(28))]+['Amount','Class']
CSV_DEFAULTS = [[0.0] for i in np.array(range(30))]+[[0]]
print('CSV_COLUMN_NAMES:{}\nCSV_DEFAULTS:{}'.format(CSV_COLUMN_NAMES,CSV_DEFAULTS))

CSV_COLUMN_NAMES:['time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
CSV_DEFAULTS:[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0]]


In [11]:
FEATURE_NAMES = CSV_COLUMN_NAMES[:-1] # all but first column

feature_cols = [tf.feature_column.numeric_column(key = k) for k in FEATURE_NAMES]
for i in feature_cols:
    print(i)  

NumericColumn(key='time', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='V10', shape=(1,), default_value=None, d

In [15]:
def serving_input_receiver_fn():
    receiver_tensors = {'time'  :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V1'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V2'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V3'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V4'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V5'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V6'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V7'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V8'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V9'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V10'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V11'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V12'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V13'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V14'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V15'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V16'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V17'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V18'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V19'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V20'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V21'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V22'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V23'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V24'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V25'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V26'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V27'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V28'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'Amount':tf.placeholder(dtype = tf.float32,shape=[None])
                        }
    features = receiver_tensors
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = receiver_tensors)


In [3]:
def parse_row(row):
    fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)
    features = dict(zip(CSV_COLUMN_NAMES, fields))
    label = features.pop("Class")
    return features, label

def read_dataset(csv_path):
    dataset = tf.data.TextLineDataset(filenames = csv_path).skip(count = 1) # skip header
    dataset = dataset.map(map_func = parse_row)
    return dataset

def train_input_fn(csv_path, batch_size = 128):
    dataset = read_dataset(csv_path)
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count = None).batch(batch_size = batch_size)
    return dataset

def eval_input_fn(csv_path, batch_size = 128):
    dataset = read_dataset(csv_path)
    dataset = dataset.batch(batch_size = batch_size)
    return dataset

In [4]:
def my_auc(labels, predictions, features):
    auc_metric = tf.keras.metrics.AUC(name="my_auc")

    #dict_keys(['probabilities', 'class_ids', 'logits', 'all_classes', 'classes', 'logistic', 'all_class_ids'])    

    auc_metric.update_state(y_true=labels, y_pred=predictions['logistic'])
    return {'auc': auc_metric}

In [5]:
def custom_metric(labels, predictions):
    print('>>>>',predictions.keys())
    predicted_classes = predictions["predictions"]  
    custom_metric = tf.metrics.mean(labels, predicted_classes, name="custom_metric")
    return {"custom_metric": custom_metric}


In [6]:
OUTDIR = "credit_trained"

config = tf.estimator.RunConfig(
    model_dir=OUTDIR,
    save_summary_steps=100,
    save_checkpoints_steps=2000
)

In [25]:
#myopt = tf.train.AdamOptimizer(learning_rate=0.01)
model = tf.estimator.DNNClassifier(
                                  #model_dir=OUTDIR,
                                  hidden_units=[10, 10],
                                  feature_columns=feature_cols,
                                  #optimizer=myopt,
                                  config = config)
#model = tf.estimator.add_metrics(model, my_auc)

INFO:tensorflow:Using config: {'_train_distribute': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_device_fn': None, '_service': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_master': '', '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9d6093d860>, '_log_step_count_steps': 100, '_model_dir': 'credit_trained', '_task_id': 0, '_protocol': None, '_save_checkpoints_steps': 2000, '_eval_distribute': None, '_task_type': 'worker', '_experimental_max_worker_delay_secs': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_save_summary_steps': 100, '_num_ps_replicas': 0}


In [27]:
!rm -r credit_trained

In [28]:
model.train(input_fn = lambda : train_input_fn('./input/creditcard_train.csv'),
    max_steps = 500
)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into credit_trained/model.ckpt.
INFO:tensorflow:loss = 802120.1, step = 1
INFO:tensorflow:global_step/sec: 35.7658
INFO:tensorflow:loss = 0.0, step = 101 (2.798 sec)
INFO:tensorflow:global_step/sec: 43.4573
INFO:tensorflow:loss = 3.9615435e-13, step = 201 (2.301 sec)
INFO:tensorflow:global_step/sec: 43.5494
INFO:tensorflow:loss = 2.4016582e-23, step = 301 (2.297 sec)
INFO:tensorflow:global_step/sec: 52.7172
INFO:tensorflow:loss = 2483.6853, step = 401 (1.896 sec)
INFO:tensorflow:Saving checkpoints for 500 into credit_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0.


In [22]:
train_spec = tf.estimator.TrainSpec(
    input_fn = lambda: train_input_fn('./input/creditcard_train.csv'),
    max_steps = 500
)
exporter = tf.estimator.FinalExporter('./exporter', serving_input_receiver_fn=serving_input_receiver_fn)
eval_spec = tf.estimator.EvalSpec(
    input_fn = lambda: eval_input_fn("./input/creditcard_eval.csv"),
    steps = None,
    start_delay_secs = 1,
    throttle_secs = 1,
    exporters = exporter,
)

In [23]:
!rm -r credit_trained

In [24]:
tf.estimator.train_and_evaluate(estimator = model, 
                                train_spec = train_spec, 
                                eval_spec = eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 2000 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into credit_trained/model.ckpt.
INFO:tensorflow:loss = 6.0003813e-10, step = 1
INFO:tensorflow:global_step/sec: 35.3398
INFO:tensorflow:loss = 0.0, step = 101 (2.832 sec)
INFO:tensorflow:global_step/sec: 42.1353
INFO:tensorflow:loss = 0.0, step = 201 (2.374 sec)
INFO:tensorflow:global_step/sec: 44.8996
INFO:tensorflow:loss = 47540.598, step = 301 (2.227 sec)
INFO:tensorflow:

({'accuracy': 0.9981461,
  'accuracy_baseline': 0.9981461,
  'auc': 0.49999297,
  'auc_precision_recall': 0.00092744024,
  'average_loss': 63.983013,
  'global_step': 500,
  'label/mean': 0.0018538805,
  'loss': 8179.028,
  'precision': 0.0,
  'prediction/mean': 3.7772713e-07,
  'recall': 0.0},
 [b'credit_trained/export/./exporter/1575870996'])